<a href="https://colab.research.google.com/github/Igor-R-Amorim/Soulcode-Academy/blob/main/Pipelines/Exerc%C3%ADcio_28_08_ApacheBeam%3DIgor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook: Igor Rocha Amorim

Problematica: <br>

O cliente quer saber, por aeroporto, a quantidade total de voos atrasados na patida do voo (aeroporto de origem), e qual o peso em minutos desses atrasos.

OBS.: Se o voo nao aconteceu, ele não está nem atrasado, nem no horario<br>
Usar o dataset original.
<br><br>

A pipeline deve entregar: um arquivo com aeroportos de origem, tempo total de voos atrasados no ano e quantas vezes foram atrasados.

#Criação de Modelo Para o Google DataFlow

Na GCP (Google Cloud Platform) modelos de pipeline no mesmo nível (projeto ou provider), tudo pode ser executado via API. Em outros níveis, é preciso fazer uma chave de autenticação.
<br><br>
Ativação  dos API's possivelmente necessários ao dataFlow para uso interno na GCP:

https://console.cloud.google.com/flows/enableapi?apiid=dataflow,compute_component,logging,storage_component,storage_api,bigquery,pubsub,datastore.googleapis.com,cloudresourcemanager.googleapis.com&_ga=2.189426155.1983962144.1622827940-955197899.1622719217&_gac=1.221249130.1622828282.Cj0KCQjwnueFBhChARIsAPu3YkSUzoSQn7RE14ObAE51mBY111345EEps_OAYCDz3smm2mLh2zjZ__8aAq9dEALw_wcB&wdLOR=cFAD8FEBF-3295-4A46-A7CC-2BD1857695D0



**Dataflow** <br>

Toda vez que criar um job, crie um nome que faça sentido.
Os jobs têm de ser únicos no projeto.
<br>

O Dataflow roda por região. Não é multiregional. É interessante definir a mesma região do restante do projeto, mas não é obrigatório.<br>
Os custos são feitos por escalonamento de VM's. Para ter uma ideia, verifique o preço da máquina por região escolhida.
<br>

# Bibliotecas

In [ ]:
# No Colab, é necessário fazer a atualização do pip, para sincronizar as versões.

!pip install --upgrade pip

Reinicie a maquina antes de dar continuidade, mesmo que não seja solicitado, a fim de evitar erros.

In [ ]:
# instala primeiro o interactive antes do gcp

#!pip install apache-beam[interactive]
!pip install apache_beam[interactive]

Reinicie novamente a maquina antes de dar continuidade, a fim de evitar erros.

In [ ]:
# não use o comando forçado com -u ou algo do tipo,
# pois isso pode retirar funções essenciais da local-machine do Colab

!pip install apache_beam[gcp]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.5/173.5 kB 17.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.7/267.7 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.1/435.1 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.6/255.6 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.1/235.1 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.2/148.2 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.2/180.2 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/1

Obedeça o aviso e reinie o runtime mais uma vez

In [ ]:
import apache_beam as beam

In [ ]:
# Conectando o Colab com o GoogleDrive que é onde mantenho minha chave json
# para acessar os os buckets dentro da gcp e onde mantenho anguns dataset's
# mais leves para estudo.

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Geração do modelo de pipeline em Batch(em lote)

### ETL a partir de DataLake


Nesse modelo, extrairemos e carregaremos os dados a partir de buckets da GCP

In [ ]:
# Diferente dos modelos executados dentro do python, precisamos passar os
# parâmetros, as opções de pipeline.
# Para isso, é preciso chamar outra parte do Apache Beam.

# Preencha com as informações da sua nuvem gcp
# project: o projeto utilizado.
# runner: quem vai ser o executor (Dataflow, neste caso).
# region: Em que região será executado.
# staging_location: cria o armazenamento de onde vão ser guardadas as coisas.
#                   Guarda os arquivos de instalação/inicialização, como se
#                   fosse um container. Ele monta as máquianas e precisa
#                   saber o que montar, quais biblis etc.
# temp_location: localização temporária. Sempre que criamos um serveless, ele
#                precisa de um cash para criar sistema
#                operacional etc (montar e executar).
# template_location: indica em qual bucket nossos
#                    templates (modelos de pipeline) serão guardados.
#                    Após o nome da pasta (models), '/modelo_bach'.

# Podemos usar o staging_location e o temp_location no mesmo
# caminho quando há poucas coisas.
#
# Quando há 100, 200 arquivos, um bucket para cada.
# Em outros providers (AWS/ Oracle/ Azure), só muda o runner. às vezes, a região

import apache_beam as beam
import os
from apache_beam.options.pipeline_options import PipelineOptions

path = 'gs://bc23-aula-igor-bucket/Pipelines-Dataflow/voos.csv'
pipeline_options = {
    'project': 'igor-aulas-bc23',
    'runner': 'DataflowRunner',
    'region': 'southamerica-east1',
    'staging_location':'gs://bc23-aula-igor-bucket/Staging/',
    'temp_location':'gs://bc23-aula-igor-bucket/Staging/',
    'template_location': 'gs://bc23-aula-igor-bucket/Pipelines-Dataflow/Modelos/modelo_batch'
    }

serviceAccount = '/content/drive/MyDrive/Acess Key/igor-aulas-bc23-f6a824308eb554866b165c671cc1c15fbb457275.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount
pipeline_options = PipelineOptions.from_dictionary(pipeline_options)


# Modelo para criar nova tabela com dados agregados de duas pipes

p1 = beam.Pipeline(options=pipeline_options)
qte_tempo = (
    p1
    |'Passo 1: Extrair do csv1'>> beam.io.ReadFromText(path, skip_header_lines = 1 )
    |'Passo 2: Separador do csv1' >> beam.Map(lambda record: record.split(','))
    |'Passo 3: 1º Filtro por aero'>> beam.Filter(lambda record: float(record[8])>0)
    |'Passo 4: 1º Agg das colunas'>> beam.Map(lambda record: (record[4],float(record[8])))
    |'Passo 5: Construir uma nova tabela por minuto' >> beam.CombinePerKey(sum)
    #|'Exibir o resultado'>> beam.Map(print)
   # |'Gravar o resultado'>> beam.io.WriteToText('voos_minutos.txt')
)

qte_vezes = (
    p1
    |'Passo 1: Extrair do csv2'>> beam.io.ReadFromText(path, skip_header_lines = 1 )
    |'Passo 2: Separador do csv2' >> beam.Map(lambda record: record.split(','))
    |'Passo 3: 2º Filtro por aero'>> beam.Filter(lambda record: float(record[8])>0)
    |'Passo 4: 2º Agg das colunas'>> beam.Map(lambda record: (record[4],float(record[8])))
    |'Passo 5: Construir uma nova tabela por qtd' >> beam.combiners.Count.PerKey()
    #|'Exibir o resultado'>> beam.Map(print)
    #|'Gravar o resultado'>> beam.io.WriteToText('voos_minutos.txt')
)

tabela = (
    {'Quantidade_minutos':qte_tempo,'Numero_de_atrasos':qte_vezes}
    |'Agrupar as pernas' >> beam.CoGroupByKey()
    #|'Resultado' >>beam.Map(print)
    |'Gravar o resultado'>> beam.io.WriteToText('gs://bc23-aula-igor-bucket/Pipelines-Dataflow/Tratado/Agregado',
                                                file_name_suffix= '.csv')
)
p1.run()

E0826 12:59:54.623848288     608 fork_posix.cc:76]           Other threads are currently calling into gRPC, skipping fork() handlers
E0826 13:00:08.750050860     608 fork_posix.cc:76]           Other threads are currently calling into gRPC, skipping fork() handlers


<DataflowPipelineResult None at 0x7fee46867250>

É conveniente ter o processamento mais próximo (em termos de região), quando em recursos diferentes. No mesmo recurso, escolha o mais barato.

O procedimento dentro do console da GCP será:

Dataflow > Create job from template
Defina o nome e a região, Escolha o modelo. Em nosso caso, Escolha Custom.

Indique o caminho do template. Aqui, podemos programar um template flex. No exemplo, não vamos utilizar e ele vai escalar conforme a necessidade.

Em 'temporary location', cole o caminho do staging (ou selecione a pasta).
Selecione o Google para encriptar as chaves.
Com as demais opções em default, clique em Run.
<br>
<br>
<br>
Um job feito não pode ser deletado.
Caso deseje fazer um novo job com o mesmo modelo, basta acessar o job feito, utilizar a função "clonar" e, se necessário, realizar as devidas alterações.

## Geração do modelo de pipeline em Streming(em tempo real)

Streaming de dados: a partir de 10 mil linhas, pode ser considerado Big Data.

Lembrando: um job feito não pode ser deletado.
Caso deseje fazer um novo job com o mesmo modelo, basta acessar o job feito, utilizar a função "clonar" e, se necessário, realizar as devidas alterações.

### Popular o topico por tempo fixo

Para utilizar o mesmo conjunto de dados, sera necessário montar <br>
uma estrutura de mensageiro através da ferramenta PUB/SUB da google

In [ ]:
pip install google-cloud-pubsub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import csv
import time
from google.cloud import pubsub_v1
import os

In [ ]:
# Data Producer (criador de dados)

# CONFIGURAÇÂO DA CHAVE DE SEGURANÇA
serviceAccount = '/content/drive/MyDrive/Acess Key/igor-aulas-bc23-f6a824308eb554866b165c671cc1c15fbb457275.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

# DETERMINAR O CAMINHO DO TOPICO PUB NA GCP
topico = 'projects/igor-aulas-bc23/topics/igorconsumer01'
publisher = pubsub_v1.PublisherClient()

# CRIAR UMA VARIÁVEL QUE VAI RECEBER O CAMINHO DO NOSSO ARQUIVO
path = '/content/drive/MyDrive/Datasets/voos.csv'

In [ ]:
# ABRIR O ARQUIVO CSV E COMECAR A ESCREVER LINHA A LINHA NO TOPICO

with open(path, 'rb') as file:
  next(file)
  for row in file:
    print('Linha enviada')
    publisher.publish(topico,row)
    #time.sleep(0.1)

### Consumir os dados do topico através do Sub

In [ ]:
# Data Consumer (Extrator de dados) via Subscription (assinatura) pub/sub

# CONFIGURAÇÂO DA CHAVE DE SEGURANÇA
serviceAccount = '/content/drive/MyDrive/Acess Key/igor-aulas-bc23-f6a824308eb554866b165c671cc1c15fbb457275.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

# DETERMINAR O CAMINHO DO SUB CRIADO NA GCP
subscription = 'projects/igor-aulas-bc23/subscriptions/igorsub02'
subscriber = pubsub_v1.SubscriberClient()


In [ ]:
# CRIANDO A FUNÇÃO DE VISUALIZAR A MENSAGEM RECEBIDA NO TOPIOC SUB
def visualizar_msg(mensagem):
  print(('mensagem:{}'.format(mensagem)))
  mensagem.ack() # INDICA QUE A MENSAGEM JÁ FOI OUVIDA PELA SUB UMA VEZ
  time.sleep(2)

# OUVINDO O SUB
subscriber.subscribe(subscription,callback=visualizar_msg)

# AGUARDANDO 1ms PARA OUVIR A PROXIMA MSG
while True:
  time.sleep(1)

# obs.: Ao rodar esse fragmento do codigo, não rode o dataflow, pois as
# mensagem já estao "ouvidas" pelo SUB. Portanto escreva novamente o
# Topico PUB

mensagem:Message {
  data: b"('BUF', {'Quantidade_minutos': [276.0], 'Numero_de..."
  ordering_key: ''
  attributes: {}
}
mensagem:Message {
  data: b"('JFK', {'Quantidade_minutos': [348.0], 'Numero_de..."
  ordering_key: ''
  attributes: {}
}
mensagem:Message {
  data: b"('FAR', {'Quantidade_minutos': [8.0], 'Numero_de_a..."
  ordering_key: ''
  attributes: {}
}
mensagem:Message {
  data: b"('LBB', {'Quantidade_minutos': [5.0], 'Numero_de_a..."
  ordering_key: ''
  attributes: {}
}
mensagem:Message {
  data: b"('BDL', {'Quantidade_minutos': [134.0], 'Numero_de..."
  ordering_key: ''
  attributes: {}
}
mensagem:Message {
  data: b"('SGF', {'Quantidade_minutos': [44.0], 'Numero_de_..."
  ordering_key: ''
  attributes: {}
}
mensagem:Message {
  data: b"('KOA', {'Quantidade_minutos': [6.0], 'Numero_de_a..."
  ordering_key: ''
  attributes: {}
}
mensagem:Message {
  data: b"('MSY', {'Quantidade_minutos': [5.0], 'Numero_de_a..."
  ordering_key: ''
  attributes: {}
}
mensagem:Message {
  dat

KeyboardInterrupt: ignored

### Modelo para operação em streaming para DataFlow

In [ ]:
# Modelo para criar nova tabela com dados agregados de duas pipes
# Modelo Streaming apra dataflowimport
from apache_beam.io.gcp.pubsub import WriteToPubSub
import apache_beam as beam
import os
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam import window

path = 'gs://bc23-aula-igor-bucket/Pipelines-Dataflow/voos.csv'

pipeline_options = {
    'project': 'igor-aulas-bc23',
    'runner': 'DataflowRunner',
    'region': 'southamerica-east1',
    'staging_location':'gs://bc23-aula-igor-bucket/Staging/',
    'temp_location':'gs://bc23-aula-igor-bucket/Staging/',
    'template_location': 'gs://bc23-aula-igor-bucket/Pipelines-Dataflow/Modelos/modelo_streaming',
    'save_main_session': True,
    'streaming':True
    }

serviceAccount = '/content/drive/MyDrive/Acess Key/igor-aulas-bc23-f6a824308eb554866b165c671cc1c15fbb457275.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

pipeline_options = PipelineOptions.from_dictionary(pipeline_options)

p1 = beam.Pipeline(options=pipeline_options)

entrada = 'projects/igor-aulas-bc23/subscriptions/igorsub01'
saida = 'projects/igor-aulas-bc23/topics/igorconsumer02'

class separador(beam.DoFn):
  def process(self,record):
    return [record.decode("utf-8").split(',')]

pcollection_entrada=(
    p1
    |'Passo1: Ler do topico1'>> beam.io.ReadFromPubSub(subscription=entrada)
)

qte_tempo = (
    pcollection_entrada
    |'Passo2: Separador do topico' >> beam.ParDo(separador())
    |'Passo3: Filtrar aeroporto'>> beam.Filter(lambda record: float(record[8])>0)
    |'Passo4: Agregar as colunas'>> beam.Map(lambda record: (record[4],float(record[8])))
    |'Passo5: Janela de Sliding'>> beam.WindowInto(window.SlidingWindows(15,10))
    |'Passo6: Construir uma nova tabela por minuto' >> beam.CombinePerKey(sum)
)

qte_vezes = (
    pcollection_entrada
    |'Passo 2: Separador do topico' >> beam.ParDo(separador())
    |'Passo 3: Filtrar aeroporto'>> beam.Filter(lambda record: float(record[8])>0)
    |'Passo 4: Agregar as colunas'>> beam.Map(lambda record: (record[4],float(record[8])))
    |'Passo 5: Janela de Sliding'>> beam.WindowInto(window.SlidingWindows(15,10))
    |'Passo 6: Construir uma nova tabela por qtd' >> beam.combiners.Count.PerKey()
)

tabela = (
    {'Quantidade_minutos':qte_tempo,'Numero_de_atrasos':qte_vezes}
    |'Agrupar as pernas' >> beam.CoGroupByKey()
    |'Converter para PubSub'>> beam.Map(lambda row:(''.join(str(row)).encode('utf-8')))
    |'Gravar o resultado'>> beam.io.WriteToPubSub(saida)
)

execucao = p1.run()
execucao.wait_until_finish()

'UNKNOWN'